미니 프로젝트

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

#카메라 인스턴스 생성

In [ ]:
import os

red_dir = "dataset2/red"
no_dir = "dataset2/no"
blue_dir = "dataset2/blue"

# 디렉토리가 이미 존재하는 경우 다음 함수가 오류를 발생시킬 수 있기 때문에 "try/except" 문을 사용.
try:
    os.makedirs(red_dir)
    os.makedirs(no_dir)
    os.makedirs(blue_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

In [ ]:
button_layout =  widgets.Layout(width='128px',height='64px')

red_button = widgets.Button(description='add RED',button_style='success',layout=button_layout)
red_button.style.button_color = 'red'

blue_button = widgets.Button(description='add BLUE',button_style='success',layout=button_layout)
blue_button.style.button_color = 'blue'

no_button = widgets.Button(description='add NO',button_style='success',layout=button_layout)
no_button.style.button_color = 'green'

red_count = widgets.IntText(layout=button_layout,value=len(os.listdir(red_dir)))
blue_count = widgets.IntText(layout=button_layout,value=len(os.listdir(blue_dir)))
no_count = widgets.IntText(layout=button_layout,value=len(os.listdir(no_dir)))

display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([blue_count, blue_button]))
display(widgets.HBox([no_count, no_button]))
# 각 사진을 저장할 수 있는 버튼과, 사진의 수량을 보여줄 텍스트 박스를 추가함.
# 미로를 구성 할 파란색, 끝을 표현할 빨간색, 아무것도 없을 때를 판단하기 위한 no.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_blue():
    global blue_dir, blue_count
    save_snapshot(blue_dir)
    blue_count.value = len(os.listdir(blue_dir))

def save_red():
    global red_dir, red_count
    save_snapshot(red_dir)
    red_count.value = len(os.listdir(red_dir))

def save_no():
    global no_dir, no_count
    save_snapshot(no_dir)
    no_count.value = len(os.listdir(no_dir))


blue_button.on_click(lambda x: save_blue())
red_button.on_click(lambda x: save_red())
no_button.on_click(lambda x: save_no())

#각 버튼 기능에 맞는 사진 저장 함수. on_Click이벤트에 등록.
#즉, 버튼 누르면 저장됨.

In [ ]:
display(image)
display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([blue_count, blue_button]))
display(widgets.HBox([no_count, no_button]))

#카메라 보여주기--> 버튼 누르면, add photo 됨.

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

dataset = datasets.ImageFolder(
    'dataset2',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

In [ ]:
model = models.alexnet(pretrained=True)

In [ ]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):

    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))

    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

실행

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224,fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)

blue_slider = widgets.FloatSlider(description='blue', min=0.0, max=1.0, orientation='vertical')
red_slider = widgets.FloatSlider(description='red', min=0.0, max=1.0, orientation='vertical')
no_slider = widgets.FloatSlider(description='no', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blue_slider,red_slider,no_slider]))

In [ ]:
from jetbot import Robot
import time

robot = Robot()

In [ ]:
import torch.nn.functional as F

def update(change):
    global blocked_slider, robot
    x = change['new']
    x = preprocess(x)
    y = model(x)

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    prob_blue = float(y.flatten()[2])
    prob_red = float(y.flatten()[1])
    prob_no = float(y.flatten()[0])

    if prob_blue < 0.6:
      return "prob_blue"
    elif prob_red < 0.6:
      return "prob_red"
    else:
      return "no"

In [1]:
def move(robot, direction):
  # 로봇을 주어진 방향으로 움직이는 함수 (예: "forward", "left", "right")
  if direction == "forward":
    robot.set_motors(1.25,1.58)
    time.sleep(0.88)
    robot.stop()

  if direction == "left":
    robot.set_motors(-1.65,1.3)
    time.sleep(0.36)
    robot.stop()

  if direction == "right":
    robot.set_motors(1.65,-1.3)
    time.sleep(0.36)
    robot.stop()


def is_valid_move(robot, direction):
    # 로봇이 주어진 방향으로 움직일 수 있는지 확인하는 함수
    # 로봇의 현재 위치와 바닥의 색상 정보를 활용하여 검사
    if update({'new': camera.value}) == "no":
            move(robot, "right")
            if update({'new': camera.value}) == "no":
                move(robot, "left")
                move(robot, "left")
                if update({'new': camera.value}) == "no":
                  move(robot, "left")
                  return True
            else:
              return True
    else:
      return True

def explore(robot):
    while True:
       # 빨간색 도착 지점에 도달한 경우, 미로 탈출 성공
        if is_valid_move(robot, "forward"):
            move(robot, "forward")

        if update({'new': camera.value}) == "prob_red":
          break


camera.observe(update, names='value')
explore(robot)

NameError: ignored